In [ ]:
def clear_projects(api):
    projects = api.projects()
    for project in projects:
        # runs = api.runs(project.name)
        # for run in runs:

        #     run.delete()

In [5]:
# 获取表现最好的config
import wandb 
import numpy as np 

api = wandb.Api()
projects = api.projects()

count = {}
# model = ['GIN', 'GCN', 'GraphSAGE', 'GTransformer']
model = ['GMT', 'DiffWire1', 'DiffWire2', 'DiffWire3', 'DiffWire4', 'EGC']
TUDataset = ['DD', 'IMDB-BINARY', 'PROTEINS', 'COLLAB', 'NCI1', 'NCI109']
OGBDataset = ['ogbg-molhiv', 'ogbg-molpcba']
best_config_dic = {}
for n in model:
    best_config_dic[n] = {}

for project in projects:
    dataset_name = project.name.split('_')[1] if len(project.name.split('_'))>1 else None
    # if dataset_name is not None and dataset_name.endswith('new'):
    #     dataset_name = dataset_name.split('new')[0]
    # if project.name.startswith('DiffWire21') or project.name.startswith('DiffWire3'):
    if project.name.startswith('EGC'):
            
        
        # print(project.name)
        if project.name.endswith('GMT'):
            print(project.name)
            model_name = project.name.split('_')[0]
            
            runs = api.runs(project.name)
            acc_list = []
            config_list = []
            for r in runs:
                config = r.config
                summary = r.summary
                # print(summary)
                if 'test_acc_of_early_stop' in summary:
                    if summary['test_acc_of_early_stop'] > 0.5:
                        acc_list.append(summary['test_acc_of_early_stop'])
                        config_list.append(config)
                else:
                    continue 
            best_index = np.argmax(acc_list)
            best_config = config_list[best_index]
            
            best_config_dic[model_name][dataset_name] = best_config
        
# runs

EGC_DD_finaltuning_GMT
EGC_NCI1_finaltuning_GMT
EGC_NCI109_finaltuning_GMT
EGC_PROTEINS_finaltuning_GMT
EGC_IMDB-BINARY_finaltuning_GMT


In [2]:
best_config_dic

{'GMT': {},
 'DiffWire1': {},
 'DiffWire2': {},
 'DiffWire3': {},
 'DiffWire4': {},
 'EGC': {'DD': {'epochs': 100,
   'dropout': 0.3,
   'reg_term': 0.1,
   'batch_size': 32,
   'num_layers': 3,
   'hidden_size': 32,
   'weight_decay': 0.001,
   'learning_rate': 0.01},
  'NCI1': {'epochs': 100,
   'dropout': 0.3,
   'reg_term': 0.1,
   'batch_size': 64,
   'num_layers': 3,
   'hidden_size': 32,
   'weight_decay': 0.01,
   'learning_rate': 0.001},
  'NCI109': {'epochs': 100,
   'dropout': 0,
   'reg_term': 0.1,
   'batch_size': 128,
   'num_layers': 3,
   'hidden_size': 32,
   'weight_decay': 0,
   'learning_rate': 0.01},
  'PROTEINS': {'epochs': 100,
   'dropout': 0,
   'reg_term': 0.1,
   'batch_size': 64,
   'num_layers': 3,
   'hidden_size': 32,
   'weight_decay': 0.01,
   'learning_rate': 0.01},
  'IMDB-BINARY': {'epochs': 100,
   'dropout': 0.3,
   'reg_term': 0.1,
   'batch_size': 128,
   'num_layers': 3,
   'hidden_size': 32,
   'weight_decay': 0.001,
   'learning_rate': 0.01}}}

In [6]:
import yaml 
import os 

for model_name, config in best_config_dic.items():
    if 'Diff' in model_name:
        continue
    print(model_name)
    filename = os.path.join('config', f"{model_name}.yaml")
    
    with open(filename, 'a+') as file:
        yaml.dump(config, file, sort_keys=False)

GMT
EGC


In [18]:
import wandb 
import numpy as np 

api = wandb.Api()
projects = api.projects()

count = {}

TUDataset = ['DD', 'IMDB-BINARY', 'PROTEINS', 'COLLAB', 'NCI1', 'NCI109']
OGBDataset = ['ogbg-molhiv', 'ogbg-molpcba']

for project in projects:
    name = project.name
    
    if (name.endswith('reg_GMT_ddl') or name.endswith('model_GMT_ddl')) and name.split('_')[1] in TUDataset:
        dataset_name = name.split('_')[1]
        if dataset_name in ['IMDB-BINARY', 'NCI1']:
            # continue
            print(name)
            # runs = api.runs(project.name)
            # for r in runs:
            #     r.delete()

GMT_NCI1_reg_GMT_ddl
GMT_NCI1_model_GMT_ddl
GMT_IMDB-BINARY_model_GMT_ddl
GMT_IMDB-BINARY_reg_GMT_ddl


In [19]:
TUDataset = ['IMDB-BINARY', 'NCI1']
cmd_list = []

for i, d in enumerate(TUDataset): 
    cmd_list.append(f'screen -dmS exp-{i} sh -c "python run_dist.py --model=GMT --dataset={d} --gpu={i}"')
    cmd_list.append(f'screen -dmS exp-{i+len(TUDataset)} sh -c "python exp_run.py --model=GMT --dataset={d} --gpu={7-i}"')
    
for i in cmd_list:
    print(i)

screen -dmS exp-0 sh -c "python run_dist.py --model=GMT --dataset=IMDB-BINARY --gpu=0"
screen -dmS exp-2 sh -c "python exp_run.py --model=GMT --dataset=IMDB-BINARY --gpu=7"
screen -dmS exp-1 sh -c "python run_dist.py --model=GMT --dataset=NCI1 --gpu=1"
screen -dmS exp-3 sh -c "python exp_run.py --model=GMT --dataset=NCI1 --gpu=6"


In [2]:
# 统计performance的均值和方差

# 获取表现最好的config
import wandb 
import numpy as np 

api = wandb.Api()
projects = api.projects()

count = {}
model = ['GIN', 'GCN', 'GraphSAGE', 'GTransformer']
TUDataset = ['DD', 'MUTAG', 'PROTEINS', 'PTC_MR', 'ENZYMES']
OGBDataset = ['ogbg-molhiv', 'ogbg-molpcba']

# for n in model:
#     count[n] = {}

for project in projects:
    dataset_name = project.name.split('_')[1] if len(project.name.split('_'))>1 else None

    if dataset_name == 'COLLAB':
        continue
    if project.name.endswith('model_GMT_ddl') and project.name.startswith('EGC'):
        # print(project.name)
        model_name = project.name.split('_')[0]
        
        runs = api.runs(project.name)
        acc_list = []
        config_list = []
        for r in runs:
            config = r.config
            summary = r.summary
            # print(summary)
            if 'test_acc_of_early_stop' in summary:
                if summary['test_acc_of_early_stop'] > 0.5:
                    acc_list.append(summary['test_acc_of_early_stop'])
                    config_list.append(config)
            else:
                continue
        acc_list = np.array(acc_list) 
        content = f'{acc_list.mean():.4}±{acc_list.std():.3}'
        count[model_name] = content
        print(f'{dataset_name}: {content}')




/tmp/ipykernel_484947/3321580412.py:41: RuntimeWarning: Mean of empty slice.
  content = f'{acc_list.mean():.4}±{acc_list.std():.3}'


ogbg-molhiv: nan±nan
NCI109: 0.7346±0.0106
NCI1: 0.7491±0.00988
PROTEINS: 0.5642±0.0726
DD: 0.7907±0.0142


### DiffWire1

带loss
IMDB-BINARY: 0.788±0.0108
PROTEINS: 0.6867±0.059
NCI109: 0.616±0.0124
NCI1: 0.6633±0.0106


不带loss
IMDB-BINARY: 0.77±0.0322
PROTEINS: 0.7117±0.0465
NCI109: 0.6208±0.0154
NCI1: 0.6594±0.0128

In [ ]:
count